In [2]:
import pandas as pd
import logging 
import numpy as np
import re 
from IPython.display import display
from sqlalchemy import MetaData, Table
from database_utils import DatabaseConnector
from data_extraction import DataExtractor

In [ ]:
# from 31 july 
# try to get to the bottom of the latitide thing 

In [3]:
# LOGGING 
print('Started cleaning_store_details')

#creating instance of dataextractor 
instance = DataExtractor()

#retrieving the data from the stores API - this takes around 90 seconds 
df = instance.retrieve_stores_data()  

Started cleaning_store_details
retrieve_stores_data is working
read_api_key is working
read_api_key is done
Skipped Stores DataFrame:
Empty DataFrame
Columns: []
Index: []


In [4]:
#making a back up of the dataframe 
df_backup = df.copy()

In [9]:
# run this to reset the df 
df = df_backup.copy()

In [10]:
# code to clean the dataframe 
# Number of rows store data before cleaning: 451
# Number of rows store data after dropping lat and cleaning latitide column: 440
# Number of rows store data after cleaning locality: 440
# Number of rows store data after cleaning continent: 440
# Number of rows store data after cleaning opening date: 428
# Number of rows store data after cleaning: 428

df.info()
num_rows = df.shape[0]
print(f"Number of rows store data before cleaning: {num_rows}")

# Clean the latitude column
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df = df.dropna(subset=['latitude'])

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning latitide column: {num_rows}")

# Dropping the 'latitude' column as it's empty 
df = df.drop(columns=['lat'])

num_rows = df.shape[0]
print(f"Number of rows store data after dropping lat column: {num_rows}")

# filtering out items in locality that aren't real place names or NULL 
pattern = r'^[a-zA-Z\s-]+$'
df = df[df['locality'].str.match(pattern)]
df['locality'] = df['locality'].replace('NULL', np.nan)
df = df.dropna(subset=['locality'])

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning locality: {num_rows}")

# replacing incorrect spellings of continents 
continent_replacements = {
    'eeEurope': 'Europe',
    'eeAmerica': 'America'
}

df['continent'] = df['continent'].replace(continent_replacements)

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning continent: {num_rows}")

# LOGGING: checking everything has worked 
#print(df['continent'].unique())
#print(df['store_type'].unique()) 
#print(df['country_code'].unique()) 
#print(df['continent'].unique())

# LOGGING: checking of datetime before is datetime64 datetype  
#is_datetime_before = pd.api.types.is_datetime64_any_dtype(df['opening_date'])

# converting opening date to datetime object 
df['opening_date'] = pd.to_datetime(df['opening_date'], format='%Y-%m-%d', errors='coerce')

# dropping any rows which contain missing values  
df = df.dropna(axis=0)

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning opening date: {num_rows}")

# LOGGING: checking the conversion worked 
#is_datetime_after = pd.api.types.is_datetime64_any_dtype(df['opening_date'])
#print(f"Is 'dates' column datetime64 dtype? {is_datetime_before}")
#print(f"Is 'dates' column datetime64 dtype? {is_datetime_after}")

num_rows = df.shape[0]
print(f"Number of rows store data after cleaning: {num_rows}")
#returning the dataframe 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          451 non-null    int64 
 1   address        451 non-null    object
 2   longitude      451 non-null    object
 3   lat            11 non-null     object
 4   locality       451 non-null    object
 5   store_code     451 non-null    object
 6   staff_numbers  451 non-null    object
 7   opening_date   451 non-null    object
 8   store_type     451 non-null    object
 9   latitude       450 non-null    object
 10  country_code   451 non-null    object
 11  continent      451 non-null    object
dtypes: int64(1), object(11)
memory usage: 42.4+ KB
Number of rows store data before cleaning: 451
Number of rows store data after cleaning latitide column: 440
Number of rows store data after dropping lat column: 440
Number of rows store data after cleaning locality: 440
Number of rows store